In [1]:
%pip install -q pyyaml h5py

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.
train_labels = train_labels[:1000]

test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.
test_labels = test_labels[:1000]

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

def create_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(28 * 28,)),
        Dropout(0.2),
        Dense(10)
    ])
    model.compile(
        optimizer='adam', 
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=SparseCategoricalAccuracy())
    return model

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint

import pathlib
import tempfile

checkpoint_root = pathlib.Path(tempfile.mkdtemp())/'checkpoints'
checkpoint_path = checkpoint_root/'training_1/cp.ckpt'
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

model = create_model()
model.fit(
    train_images, train_labels, epochs=10, 
    validation_data=(test_images, test_labels),
    callbacks=[checkpoint_callback])

Epoch 1/10
 1/32 [..............................] - ETA: 6s - loss: 2.4254 - sparse_categorical_accuracy: 0.0625
Epoch 00001: saving model to /tmp/tmp3923cs_k/checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 1.1523 - sparse_categorical_accuracy: 0.6590 - val_loss: 0.7728 - val_sparse_categorical_accuracy: 0.7530
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.5649 - sparse_categorical_accuracy: 0.8125
Epoch 00002: saving model to /tmp/tmp3923cs_k/checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.4379 - sparse_categorical_accuracy: 0.8790 - val_loss: 0.5265 - val_sparse_categorical_accuracy: 0.8310
Epoch 3/10
 1/32 [..............................] - ETA: 0s - loss: 0.3456 - sparse_categorical_accuracy: 0.9375
Epoch 00003: saving model to /tmp/tmp3923cs_k/checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.2947 - sparse_categorical_accurac

In [22]:
model = create_model()
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f'Untrained model: loss {loss:.2f} accuracy {accuracy:.2f}')

model.load_weights(checkpoint_path)
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model: loss {loss:.2f} accuracy {accuracy:.2f}')

32/32 - 0s - loss: 2.2939 - sparse_categorical_accuracy: 0.1070 - 88ms/epoch - 3ms/step
Untrained model: loss 2.29 accuracy 0.11
32/32 - 0s - loss: 0.4078 - sparse_categorical_accuracy: 0.8620 - 37ms/epoch - 1ms/step
Restored model: loss 0.41 accuracy 0.86


In [23]:
batch_size = 32

checkpoint_path2 = str(checkpoint_root/'training_2/cp-{epoch:04d}.ckpt')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path2, 
    save_weights_only=True, save_freq=5 * batch_size, 
    verbose=1)

model = create_model()
model.save_weights(checkpoint_path2.format(epoch=0))
model.fit(
    train_images, train_labels, epochs=50, batch_size=batch_size,
    validation_data=(test_images, test_labels),
    callbacks=[checkpoint_callback],
    verbose=0)


Epoch 00005: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0005.ckpt

Epoch 00010: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0010.ckpt

Epoch 00015: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0015.ckpt

Epoch 00020: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0020.ckpt

Epoch 00025: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0025.ckpt

Epoch 00030: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0030.ckpt

Epoch 00035: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0035.ckpt

Epoch 00040: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0040.ckpt

Epoch 00045: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0045.ckpt

Epoch 00050: saving model to /tmp/tmp3923cs_k/checkpoints/training_2/cp-0050.ckpt


In [24]:
from tensorflow import train

import os

checkpoint_directory2 = os.path.dirname(checkpoint_path2)
latest = train.latest_checkpoint(checkpoint_directory2)

model = create_model()
model.load_weights(latest)
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model: loss {loss:.2f} accuracy {accuracy:.2f}')

32/32 - 0s - loss: 0.5020 - sparse_categorical_accuracy: 0.8730 - 87ms/epoch - 3ms/step
Restored model: loss 0.50 accuracy 0.87


In [29]:
from tensorflow.keras.models import load_model

saved_model_root = pathlib.Path(tempfile.mkdtemp() + '-models')
saved_model_path = str(saved_model_root/'training_2')
model.save(saved_model_path)

model = load_model(saved_model_path)
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model: loss {loss:.2f} accuracy {accuracy:.2f}')

INFO:tensorflow:Assets written to: /tmp/tmpjcl0omdg-models/training_2/assets
32/32 - 0s - loss: 0.5020 - sparse_categorical_accuracy: 0.8730 - 89ms/epoch - 3ms/step
Restored model: loss 0.50 accuracy 0.87


In [30]:
saved_model_path2 = str(saved_model_root/'training_3/model.h5')
model.save(saved_model_path2)

model = load_model(saved_model_path2)
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model: loss {loss:.2f} accuracy {accuracy:.2f}')

32/32 - 0s - loss: 0.5020 - sparse_categorical_accuracy: 0.8730 - 88ms/epoch - 3ms/step
Restored model: loss 0.50 accuracy 0.87
